# Landmarks Path Computation
In this notebook, we train emotion models using the CK+ dataset.

The idea is basicaly to compute the normalized landmarks paths over a sequence of images.

These landmarks are provided by the dataset, where it was computed using AAM.

To achieve normalization, it is computed the distance of each landmark point to a reference point (nose), and it is diveded by the distance between eyes.

In [1]:
# Database parser
getLandmarkDots(filename) = map((x)->parse(x), split(readall(filename), ['\n', ' '], limit=0, keep=false));
getLandmarkPoint(id, dots) = [dots[id*2-1], dots[id*2]];

In [2]:
dots = getLandmarkDots("/home/data/ckplus/Landmarks/S100/001/S100_001_00000001_landmarks.txt")

136-element Array{Float64,1}:
 181.333
 191.839
 183.712
 221.229
 186.614
 250.323
 190.084
 279.266
 199.901
 306.639
 218.007
 329.651
 242.417
   ⋮    
 297.656
 283.435
 313.561
 284.371
 328.846
 288.721
 313.387
 285.161
 297.658
 284.29 
 282.178
 282.546

In [3]:
# Reference Point: nose
referenceID = 34
referencePoint = getLandmarkPoint(referenceID, dots)

2-element Array{Float64,1}:
 297.659
 248.387

In [4]:
# Reference Size: distance between eyes
using Distances
referenceSize = evaluate(Euclidean(), getLandmarkPoint(40, dots), getLandmarkPoint(43, dots))
println(referenceSize)
if referenceSize < 1.0 error("Distance between eyes less than one.") end

67.

In [7]:
# Normalized vector
normalizedVector = []
for id = 1:round(Int,(length(dots) / 2))
    if id == referenceID; continue end
    append!(normalizedVector, (getLandmarkPoint(id, dots) - referencePoint) / referenceSize)
end
show(normalizedVector)

Any[-1.7158394999072653,-0.8341003087650835,-1.6807565117013892,-0.4005991496374491,-1.6379408285008328,0.028543887135675343,-1.5867605476396938,0.45546302183275067,-1.4419571901910422,0.8592302399711864,-1.1748840594584802,1.1986616835588455,-0.8148345050999629,1.429212787241336,-0.4201478639732493,1.6038141019623333,0.0021087022669763617,1.667898236469219,0.43672660627872373,1.6297935026952635,0.8515625937803495,1.5073530684906136,1.2234370176573808,1.2913125681596742,1.5011774147053765,0.9569811867980197,1.7052554252593315,0.5788240219892891,1.8251622011791573,0.1590772185549247,1.9293377867819008,-0.2625661778948834,1.9721381296749751,-0.6884610520616086,-1.3683294668917931,-1.5332833507390093,-1.137917606000295,-1.649639730494445,-0.8839309375737012,-1.6868697717360026,-0.6248470094772772,-1.6712726616064333,-0.3822157086584681,-1.5895833117193754,0.6079614610221604,-1.5939219636836197,0.8622758810182295,-1.6201816200340131,1.1147068357619894,-1.6118410653553503,1.3589706993040003

In [8]:
using RegERMs